<a href="https://colab.research.google.com/github/amilafr/algo-python-pro2/blob/main/M4L2_Strategy_and_tactics_for_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Strategy and tactics for model training.


1. Build a mathematical model that will predict whether a user of the DigitalEdu portal purchases a course. The data set contains various characteristics for each user, such as gender, date of birth, study format, etc., as well as a target variable — the result (1 if the user bought the course; 0 if they did not).

In [46]:
# get dataset
import pandas as pd

df = pd.read_csv('train.csv')

df.head()

,id,sex,bdate,has_photo,has_mobile,followers_count,graduation,education_form,relation,education_status,langs,life_main,people_main,city,last_seen,occupation_type,occupation_name,career_start,career_end,result
0,64100,2,23.4.1990,1,1.0,150.0,2012.0,Full-time,2.0,Student (Specialist),Русский;English,0,0,Kaliningrad,2020-08-19 20:47:43,university,"БФУ им. Канта (бывш. РГУ им. Канта, КГУ)",False,False,0
1,46851,2,11.4,0,1.0,92.0,2011.0,Distance Learning,4.0,Alumnus (Specialist),Русский,0,0,NaN,2020-12-25 10:31:53,university,ЛНУ (бывш. ЛНПУ) им. Тараса Шевченко,False,False,0
2,13159,2,5.10,0,1.0,62.0,2012.0,Full-time,0.0,Alumnus (Specialist),Русский,0,6,Mogilev,2020-03-06 03:37:47,university,"БРУ (бывш. МГТУ, ММИ)",False,False,0
3,59148,1,1.3.1967,0,1.0,179.0,1989.0,Full-time,4.0,PhD,Русский,False,False,Moscow,2020-10-10 18:06:27,university,Bodø Graduate School of Business,False,False,0
4,46429,2,NaN,0,1.0,74.0,2008.0,Full-time,7.0,Alumnus (Specialist),Русский,3,2,Ryazan,2020-01-03 03:34:09,university,"РГРТУ (бывш. РГРТА, РРТИ)",False,False,0


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8193 non-null   int64  
 1   sex               8193 non-null   int64  
 2   bdate             7025 non-null   object 
 3   has_photo         8193 non-null   int64  
 4   has_mobile        8193 non-null   float64
 5   followers_count   8193 non-null   float64
 6   graduation        8193 non-null   float64
 7   education_form    7585 non-null   object 
 8   relation          8193 non-null   float64
 9   education_status  8193 non-null   object 
 10  langs             8193 non-null   object 
 11  life_main         8193 non-null   object 
 12  people_main       8193 non-null   object 
 13  city              7611 non-null   object 
 14  last_seen         8193 non-null   object 
 15  occupation_type   8000 non-null   object 
 16  occupation_name   8000 non-null   object 


## Hypothesis

In [48]:
df.groupby('education_form')['result'].mean()

,result
education_form,
Distance Learning,0.453456
Full-time,0.541645
Part-time,0.478125


In [49]:
df.groupby('education_status')['result'].mean()

,result
education_status,
Alumnus (Bachelor's),0.635688
Alumnus (Master's),0.640756
Alumnus (Specialist),0.573306
Candidate of Sciences,0.600000
PhD,0.234432
Student (Bachelor's),0.558442
Student (Master's),0.577540
Student (Specialist),0.462453
Undergraduate applicant,0.236025


In [50]:
df.pivot_table(
    index = 'education_status',
    columns = 'education_form',
    values = 'result',
    aggfunc = 'mean'
)

education_form,Distance Learning,Full-time,Part-time
education_status,,,
Alumnus (Bachelor's),0.507353,0.648452,0.483871
Alumnus (Master's),0.520000,0.625698,0.466667
Alumnus (Specialist),0.548757,0.555824,0.626866
Candidate of Sciences,0.500000,0.619718,0.200000
PhD,0.203125,0.211180,0.227273
Student (Bachelor's),0.358824,0.596947,0.411765
Student (Master's),0.440000,0.585185,0.500000
Student (Specialist),0.413534,0.475692,0.358209
Undergraduate applicant,0.191489,0.240000,0.125000


In [51]:
df.groupby('occupation_type')['result'].mean()

,result
occupation_type,
university,0.547551
work,0.531588


In [52]:
df.pivot_table(
    index = 'education_status',
    columns = ['education_form', 'occupation_type'],
    values = 'result',
    aggfunc = 'mean'
)

education_form          Distance Learning            Full-time            \
occupation_type                university      work university      work   
education_status                                                           
Alumnus (Bachelor's)             0.551724  0.416667   0.644836  0.657718   
Alumnus (Master's)               0.454545  0.666667   0.611570  0.657407   
Alumnus (Specialist)             0.535211  0.562500   0.557984  0.553325   
Candidate of Sciences            0.666667  0.000000   0.605263  0.677419   
PhD                              0.264706  0.115385   0.240385  0.186047   
Student (Bachelor's)             0.370370  0.350000   0.606122  0.565217   
Student (Master's)               0.526316  0.200000   0.556701  0.657143   
Student (Specialist)             0.400735  0.440678   0.504292  0.420904   
Undergraduate applicant          0.142857  0.235294   0.281250  0.192308   

education_form           Part-time            
occupation_type         university      work  
education_status                              
Alumnus (Bachelor's)      0.500000  0.500000  
Alumnus (Master's)        0.555556  0.333333  
Alumnus (Specialist)      0.585859  0.764706  
Candidate of Sciences     0.000000  0.000000  
PhD                       0.142857  0.285714  
Student (Bachelor's)      0.423077  0.375000  
Student (Master's)        0.500000  0.500000  
Student (Specialist)      0.391304  0.285714  
Undergraduate applicant   0.000000  0.333333

## Clean Data

In [53]:
'''
hapus kolom yang gak perlu (tidak mempengaruhi untuk ambil course)
id, bdate, langs, city, last seen, occupation_name, life_main, people_main, career_start, career_end*
*bisa disesuaikan sendiri
'''

df.drop(['id', 'bdate', 'langs', 'city', 'last_seen', 'occupation_name', 'life_main', 'people_main', 'career_start', 'career_end'], axis = 1, inplace = True)

In [54]:
df.head()

,sex,has_photo,has_mobile,followers_count,graduation,education_form,relation,education_status,occupation_type,result
0,2,1,1.0,150.0,2012.0,Full-time,2.0,Student (Specialist),university,0
1,2,0,1.0,92.0,2011.0,Distance Learning,4.0,Alumnus (Specialist),university,0
2,2,0,1.0,62.0,2012.0,Full-time,0.0,Alumnus (Specialist),university,0
3,1,0,1.0,179.0,1989.0,Full-time,4.0,PhD,university,0
4,2,0,1.0,74.0,2008.0,Full-time,7.0,Alumnus (Specialist),university,0


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sex               8193 non-null   int64  
 1   has_photo         8193 non-null   int64  
 2   has_mobile        8193 non-null   float64
 3   followers_count   8193 non-null   float64
 4   graduation        8193 non-null   float64
 5   education_form    7585 non-null   object 
 6   relation          8193 non-null   float64
 7   education_status  8193 non-null   object 
 8   occupation_type   8000 non-null   object 
 9   result            8193 non-null   int64  
dtypes: float64(4), int64(3), object(3)
memory usage: 640.2+ KB


### fill NaN

In [56]:
df['education_form'].value_counts()

,count
education_form,
Full-time,5847
Distance Learning,1418
Part-time,320


In [57]:
df['education_form'].fillna('Full-time', inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sex               8193 non-null   int64  
 1   has_photo         8193 non-null   int64  
 2   has_mobile        8193 non-null   float64
 3   followers_count   8193 non-null   float64
 4   graduation        8193 non-null   float64
 5   education_form    8193 non-null   object 
 6   relation          8193 non-null   float64
 7   education_status  8193 non-null   object 
 8   occupation_type   8000 non-null   object 
 9   result            8193 non-null   int64  
dtypes: float64(4), int64(3), object(3)
memory usage: 640.2+ KB


In [58]:
df['occupation_type'].value_counts()

,count
occupation_type,
university,5594
work,2406


In [59]:
df['occupation_type'].fillna('university', inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sex               8193 non-null   int64  
 1   has_photo         8193 non-null   int64  
 2   has_mobile        8193 non-null   float64
 3   followers_count   8193 non-null   float64
 4   graduation        8193 non-null   float64
 5   education_form    8193 non-null   object 
 6   relation          8193 non-null   float64
 7   education_status  8193 non-null   object 
 8   occupation_type   8193 non-null   object 
 9   result            8193 non-null   int64  
dtypes: float64(4), int64(3), object(3)
memory usage: 640.2+ KB


### change values

In [60]:
df.head()

,sex,has_photo,has_mobile,followers_count,graduation,education_form,relation,education_status,occupation_type,result
0,2,1,1.0,150.0,2012.0,Full-time,2.0,Student (Specialist),university,0
1,2,0,1.0,92.0,2011.0,Distance Learning,4.0,Alumnus (Specialist),university,0
2,2,0,1.0,62.0,2012.0,Full-time,0.0,Alumnus (Specialist),university,0
3,1,0,1.0,179.0,1989.0,Full-time,4.0,PhD,university,0
4,2,0,1.0,74.0,2008.0,Full-time,7.0,Alumnus (Specialist),university,0


Education Form

In [61]:
# education form
# create dummy variables
pd.get_dummies(df['education_form'])

,Distance Learning,Full-time,Part-time
0,False,True,False
1,True,False,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
8188,False,False,True
8189,False,True,False
8190,False,True,False
8191,True,False,False


In [62]:
df[list(pd.get_dummies(df['education_form']).columns)] = pd.get_dummies(df['education_form'])

df.head()

,sex,has_photo,has_mobile,followers_count,graduation,education_form,relation,education_status,occupation_type,result,Distance Learning,Full-time,Part-time
0,2,1,1.0,150.0,2012.0,Full-time,2.0,Student (Specialist),university,0,False,True,False
1,2,0,1.0,92.0,2011.0,Distance Learning,4.0,Alumnus (Specialist),university,0,True,False,False
2,2,0,1.0,62.0,2012.0,Full-time,0.0,Alumnus (Specialist),university,0,False,True,False
3,1,0,1.0,179.0,1989.0,Full-time,4.0,PhD,university,0,False,True,False
4,2,0,1.0,74.0,2008.0,Full-time,7.0,Alumnus (Specialist),university,0,False,True,False


In [63]:
df.drop('education_form', axis = 1, inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sex                8193 non-null   int64  
 1   has_photo          8193 non-null   int64  
 2   has_mobile         8193 non-null   float64
 3   followers_count    8193 non-null   float64
 4   graduation         8193 non-null   float64
 5   relation           8193 non-null   float64
 6   education_status   8193 non-null   object 
 7   occupation_type    8193 non-null   object 
 8   result             8193 non-null   int64  
 9   Distance Learning  8193 non-null   bool   
 10  Full-time          8193 non-null   bool   
 11  Part-time          8193 non-null   bool   
dtypes: bool(3), float64(4), int64(3), object(2)
memory usage: 600.2+ KB


Education Status

In [64]:
df[list(pd.get_dummies(df['education_status']).columns)] = pd.get_dummies(df['education_status'])

df.head()

,sex,has_photo,has_mobile,followers_count,graduation,relation,education_status,occupation_type,result,Distance Learning,...,Part-time,Alumnus (Bachelor's),Alumnus (Master's),Alumnus (Specialist),Candidate of Sciences,PhD,Student (Bachelor's),Student (Master's),Student (Specialist),Undergraduate applicant
0,2,1,1.0,150.0,2012.0,2.0,Student (Specialist),university,0,False,...,False,False,False,False,False,False,False,False,True,False
1,2,0,1.0,92.0,2011.0,4.0,Alumnus (Specialist),university,0,True,...,False,False,False,True,False,False,False,False,False,False
2,2,0,1.0,62.0,2012.0,0.0,Alumnus (Specialist),university,0,False,...,False,False,False,True,False,False,False,False,False,False
3,1,0,1.0,179.0,1989.0,4.0,PhD,university,0,False,...,False,False,False,False,False,True,False,False,False,False
4,2,0,1.0,74.0,2008.0,7.0,Alumnus (Specialist),university,0,False,...,False,False,False,True,False,False,False,False,False,False


In [65]:
df.drop('education_status', axis = 1, inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sex                      8193 non-null   int64  
 1   has_photo                8193 non-null   int64  
 2   has_mobile               8193 non-null   float64
 3   followers_count          8193 non-null   float64
 4   graduation               8193 non-null   float64
 5   relation                 8193 non-null   float64
 6   occupation_type          8193 non-null   object 
 7   result                   8193 non-null   int64  
 8   Distance Learning        8193 non-null   bool   
 9   Full-time                8193 non-null   bool   
 10  Part-time                8193 non-null   bool   
 11  Alumnus (Bachelor's)     8193 non-null   bool   
 12  Alumnus (Master's)       8193 non-null   bool   
 13  Alumnus (Specialist)     8193 non-null   bool   
 14  Candidate of Sciences   

Occupation Type

In [66]:
# occupation_type
df[list(pd.get_dummies(df['occupation_type']).columns)] = pd.get_dummies(df['occupation_type'])
df.drop('occupation_type', axis = 1, inplace = True)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sex                      8193 non-null   int64  
 1   has_photo                8193 non-null   int64  
 2   has_mobile               8193 non-null   float64
 3   followers_count          8193 non-null   float64
 4   graduation               8193 non-null   float64
 5   relation                 8193 non-null   float64
 6   result                   8193 non-null   int64  
 7   Distance Learning        8193 non-null   bool   
 8   Full-time                8193 non-null   bool   
 9   Part-time                8193 non-null   bool   
 10  Alumnus (Bachelor's)     8193 non-null   bool   
 11  Alumnus (Master's)       8193 non-null   bool   
 12  Alumnus (Specialist)     8193 non-null   bool   
 13  Candidate of Sciences    8193 non-null   bool   
 14  PhD                     

In [67]:
df.head()

,sex,has_photo,has_mobile,followers_count,graduation,relation,result,Distance Learning,Full-time,Part-time,...,Alumnus (Master's),Alumnus (Specialist),Candidate of Sciences,PhD,Student (Bachelor's),Student (Master's),Student (Specialist),Undergraduate applicant,university,work
0,2,1,1.0,150.0,2012.0,2.0,0,False,True,False,...,False,False,False,False,False,False,True,False,True,False
1,2,0,1.0,92.0,2011.0,4.0,0,True,False,False,...,False,True,False,False,False,False,False,False,True,False
2,2,0,1.0,62.0,2012.0,0.0,0,False,True,False,...,False,True,False,False,False,False,False,False,True,False
3,1,0,1.0,179.0,1989.0,4.0,0,False,True,False,...,False,False,False,True,False,False,False,False,True,False
4,2,0,1.0,74.0,2008.0,7.0,0,False,True,False,...,False,True,False,False,False,False,False,False,True,False


In [68]:
# df['career_start'].value_counts()

## Build KNN

In [69]:
# A function for splitting the original dataset into samples
# for training and testing the model.
from sklearn.model_selection import train_test_split

# Class for standardizing metrics.
from sklearn.preprocessing import StandardScaler

# Class for creating and training the model.
from sklearn.neighbors import KNeighborsClassifier

# Functions for evaluating the accuracy of the model.
from sklearn.metrics import confusion_matrix, accuracy_score

In [70]:
'''Divide data --> x y'''

x = df.drop('result', axis = 1) # data
y = df['result'] # Target variable

In [71]:
'''Split data --> train dan test (80% train, 20% test)'''

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [72]:
'''standardize metrics'''

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [73]:
'''KNN classifier'''
classifier = KNeighborsClassifier(n_neighbors = 3) # jumlah neighbours = 3

In [74]:
'''Train'''

classifier.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [75]:
'''Predict'''

y_pred = classifier.predict(x_test)

In [76]:
'''Evaluate'''

percent = accuracy_score(y_test, y_pred) * 100

print(percent)

78.58450274557657


In [77]:
'''confusion matrix'''

print(confusion_matrix(y_test, y_pred))

[[606 147]
 [204 682]]
